In [ ]:
!pip3 install -U tensorflow

In [16]:
import tensorflow as tf
import numpy as np

print('TensorFlow:', tf.__version__)

drive.mount

TensorFlow: 2.13.0


In [17]:
# 변경 불가능한 Tensor 생성
t1 = tf.constant(42)
print(t1)

t2 = tf.constant([[1, 2, 3], [4, 5, 6]])
print(t2)

tf.Tensor(42, shape=(), dtype=int32)
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [18]:
print(t2[:, 1:]) # 행 전체 그리고 열은 1번 부터 끝까지 선택
print(t2[:, 1]) # 첫 번째 열 - 1차원 배열

# tf.newaxis를 추가하면 차원이 하나 추가됨. (reshape 사용하지 않아도 됨)
print(t2[:, 1, tf.newaxis]) # 첫 번째 열 데이터 - 2차원 배열


tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor([2 5], shape=(2,), dtype=int32)
tf.Tensor(
[[2]
 [5]], shape=(2, 1), dtype=int32)


In [19]:
print(t2 + 10) # broadcast 연산 수행
print(tf.square(t2))
print(t2 @ tf.transpose(t2))

tf.Tensor(
[[11 12 13]
 [14 15 16]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)


In [20]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t2)) + 10

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[11, 26],
       [14, 35],
       [19, 46]], dtype=int32)>

In [22]:
a = np.array([2., 4., 5.])
print(tf.constant(a))


tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)


In [27]:
# InvalidArgumentError 자료형이 달라서 에러 발생
# 묵시적 형 변환을 지원하지 않음.
try:
    print(tf.cast(tf.constant(10), tf.float32) + tf.constant(10.3))

except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.Tensor(20.3, shape=(), dtype=float32)


In [30]:
v = tf.Variable([[1, 2, 3], [4, 5, 6]])
print(v)
v.assign(5 + v)
print(v)

v[0, 0].assign(42)
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[ 6,  7,  8],
       [ 9, 10, 11]], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[42,  7,  8],
       [ 9, 10, 11]], dtype=int32)>


# 문자열 Tensor

In [32]:
tf.constant('안녕하세요')

# <tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'>

<tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'>

In [53]:
# Tensor 배열 - 실수 3개를 저장할 수 있는 텐서 배열을 생성
array = tf.TensorArray(dtype=tf.float32, size=3)
array = array.write(0, tf.constant([1.1, 2.2]))
array = array.write(1, tf.constant([3.3, 4.4]))
array = array.write(2, tf.constant([5.5, 6.6]))

print(array.read(2))

tf.Tensor([5.5 6.6], shape=(2,), dtype=float32)


In [54]:
print(array.stack())

tf.Tensor(
[[1.1 2.2]
 [3.3 4.4]
 [0.  0. ]], shape=(3, 2), dtype=float32)


In [56]:
# ragged tensor: 각 배열의 길이가 달라도 됨. 자료형이 다른 것은 안 됨.
ar = tf.ragged.constant([[10, 20], [30, 40, 50], [50, 60]])
print(ar)

<tf.RaggedTensor [[10, 20], [30, 40, 50], [50, 60]]>


In [61]:
# 희소 텐서 생성
# 3행 4열의 행렬을 만들고 indices 자리에 values의 값을 순서대로 배정한다.

s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]],
                    values = [1.0, 2.0, 3.0],
                    dense_shape=[3, 4])

print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [70]:
s1 = tf.constant([[1, 2, 3, 1, 2, 3], [2, 4, 5, 8, 1, 2]])
s2 = tf.constant([[1, 2, 3, 4, 5, 2], [2, 7, 9, 0, 1, 2]])

print(s1)
print(s2)
print(tf.sparse.to_dense(tf.sets.union(s1, s2)))

# set 연산은 배열로 리턴하기 때문에 연산의 결과 각 배열의 데이터 개수가 다르면 뒤에 0을 채워서
# 동일한 개수로 맞춰준다
# 1행은 합집합하면 데이터 4가지가 되고 2행은 7가지가 되므로 1행의 결과 뒤에 0을 3개 추가

# tf.Tensor(
# [[1 2 3 4 5 0 0 0]
#  [0 1 2 4 5 7 8 9]], shape=(2, 8), dtype=int32)

tf.Tensor(
[[1 2 3 1 2 3]
 [2 4 5 8 1 2]], shape=(2, 6), dtype=int32)
tf.Tensor(
[[1 2 3 4 5 2]
 [2 7 9 0 1 2]], shape=(2, 6), dtype=int32)
tf.Tensor(
[[1 2 3 4 5 0 0 0]
 [0 1 2 4 5 7 8 9]], shape=(2, 8), dtype=int32)


In [71]:
node1 = tf.constant(3.0)
node2 = tf.constant(4.0)

print(node1)
print(node2)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [73]:
# 텐서플로우 코드 최적화가 가능하도록 작성
@tf.function
def tf_cube(x):
    print(x)
    return x ** 3

result = tf_cube(tf.constant(2.0))
print(result)

Tensor("x:0", shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


In [76]:
# 2행 5열의 난수 발생: 0애서 1사이의 균등한 분포
result = tf.random.uniform([2, 5], 0.0, 1.0)
print(result)
# 2행 5열의 난수 발생: 0에서 1사이의 정규 분포, 평균은 0, 표준 편차는 1
reslut = tf.random.normal([2, 5], 0.0, 1.0)
print(result)


tf.Tensor(
[[0.46540916 0.8346325  0.08375227 0.8615142  0.67296994]
 [0.34541142 0.21023452 0.6014104  0.33998454 0.98780775]], shape=(2, 5), dtype=float32)
tf.Tensor(
[[0.46540916 0.8346325  0.08375227 0.8615142  0.67296994]
 [0.34541142 0.21023452 0.6014104  0.33998454 0.98780775]], shape=(2, 5), dtype=float32)


In [ ]:
#